In [1]:
#import libraries 
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

#import data file
sample = pd.read_csv("original_data.csv")

In [2]:
sample.head()

,Unnamed: 0,object_id,line_number,description,value,name
0,0,202000949349300500,Part IX Line 1; col (A),Grants to governments and organizations in the...,290690,Virginia Gay Hospital Health Care
1,1,202000949349300500,Part IX Line 25; col (A),Total functional expenses; Total,290690,Virginia Gay Hospital Health Care
2,2,202000949349300500,Part IX Line 25; col (B),Total functional expenses; Program services,290690,Virginia Gay Hospital Health Care
3,3,202000949349300500,Part IX Line 25; col (C),Total functional expenses; Management and general,0,Virginia Gay Hospital Health Care
4,4,202000949349300500,Part IX Line 25; col (D),Total functional expenses; Fundraising,0,Virginia Gay Hospital Health Care


In [3]:
#get list of descriptions
sample['description'].value_counts()

Other expenses; Total                                                                                                            81595
Total functional expenses; Total                                                                                                 25168
Total functional expenses; Program services                                                                                      25165
Total functional expenses; Management and general                                                                                25165
Total functional expenses; Fundraising                                                                                           25165
Office expenses; Total                                                                                                           22017
Depreciation, depletion, and amortization; Total                                                                                 19883
Fees for services (non-employees): Other; Total        

In [4]:
#drop irrelevant rows
sample.drop(sample[sample['description']=="Total book value"].index, inplace=True)
sample.drop(sample[sample['description']=="Total joint costs; Program services"].index, inplace=True)
sample.drop(sample[sample['description']=="Total joint costs; Fundraising"].index, inplace=True)
sample.drop(sample[sample['description']=="Total joint costs; Management and general"].index, inplace=True)
sample.drop(sample[sample['description']=="Total functional expenses; Program services"].index, inplace=True)
sample.drop(sample[sample['description']=="Total functional expenses; Management and general"].index, inplace=True)
sample.drop(sample[sample['description']=="Total functional expenses; Fundraising"].index, inplace=True)


In [5]:
#check data
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 557444 entries, 0 to 640887
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   557444 non-null  int64 
 1   object_id    557444 non-null  int64 
 2   line_number  557444 non-null  object
 3   description  557444 non-null  object
 4   value        557444 non-null  int64 
 5   name         557444 non-null  object
dtypes: int64(3), object(3)
memory usage: 29.8+ MB


In [6]:
#check for unique lines
sorted(sample['line_number'].unique())

['Part IX Line 10; col (A)',
 'Part IX Line 11a; col (A)',
 'Part IX Line 11b; col (A)',
 'Part IX Line 11c; col (A)',
 'Part IX Line 11d; col (A)',
 'Part IX Line 11e; col (A)',
 'Part IX Line 11f; col (A)',
 'Part IX Line 11g; col (A)',
 'Part IX Line 12; col (A)',
 'Part IX Line 13; col (A)',
 'Part IX Line 14; col (A)',
 'Part IX Line 15; col (A)',
 'Part IX Line 16; col (A)',
 'Part IX Line 17; col (A)',
 'Part IX Line 18; col (A)',
 'Part IX Line 19; col (A)',
 'Part IX Line 1; col (A)',
 'Part IX Line 20; col (A)',
 'Part IX Line 21; col (A)',
 'Part IX Line 22; col (A)',
 'Part IX Line 23; col (A)',
 'Part IX Line 24a - 24d; col (A)',
 'Part IX Line 24a - 24e; col (A)',
 'Part IX Line 24e; col (A)',
 'Part IX Line 24f; col (A)',
 'Part IX Line 25; col (A)',
 'Part IX Line 26; col (A)',
 'Part IX Line 2; col (A)',
 'Part IX Line 3; col (A)',
 'Part IX Line 4; col (A)',
 'Part IX Line 5; col (A)',
 'Part IX Line 6; col (A)',
 'Part IX Line 7; col (A)',
 'Part IX Line 8; col (A)',

In [7]:
#get all rows for Line 24a - 24d
test_24ad = sample[sample['line_number']=='Part IX Line 24a - 24d; col (A)']
#get all rows for Line 24e
test_24e = sample[sample['line_number']=='Part IX Line 24e; col (A)']

# add all the values for 24a to 24e
for i in test_24e['object_id']:
    #set count to zero for each object_id
    count = 0 
    #add count of Line 24e
    count = count + test_24e['value'].where(test_24e['object_id']==i).sum()
    #add count of Line 24a-d (4 lines)
    count = count + test_24ad['value'].where(test_24ad['object_id']==i).sum()
    #get index of object_id
    i_index = test_24e[test_24e['object_id']==i].index
    #update sample with total value at Line 24e
    sample.at[i_index,'value'] = count
    #rename line 24e to Line 24com
    sample.at[i_index,'line_number'] = 'Part IX Line 24com'

#drop Line 24a - 24d for rows with added sum in Line 24e
for i in test_24e['object_id']:
    loc_24 = sample[sample['object_id']==i]
    loc_index = loc_24[loc_24['line_number'] =='Part IX Line 24a - 24d; col (A)'].index
    sample.drop(loc_index, inplace=True)
         

In [8]:
#get all rows for Line 24a - 24d that did not have 24e
test_24adNOTe = sample[sample['line_number']=='Part IX Line 24a - 24d; col (A)']

# add all the values for 24a to 24d
for i in test_24adNOTe['object_id']:
    #set count to zero for each object_id
    count = 0 
    #add count of Line 24a-d 
    count = count + test_24adNOTe['value'].where(test_24adNOTe['object_id']==i).sum()
    #get index of object_id
    i_index = test_24adNOTe[test_24adNOTe['object_id']==i].index
    #update sample with total value
    sample.at[i_index,'value'] = count
    #change the name of the file line to Line 24com
    sample.at[i_index[0],'line_number'] = 'Part IX Line 24com'


In [9]:
#get list of Line 24com
test_24com = sample[sample['line_number']=='Part IX Line 24com']

#drop Line 24a - 24d for rows with added sum in Line 24com
for i in test_24com['object_id']:
    l_24 = sample[sample['object_id']==i]
    l_index = l_24[l_24['line_number'] =='Part IX Line 24a - 24d; col (A)'].index
    sample.drop(l_index, inplace=True)


In [10]:
#get all rows for Line 24a - 24e
test_24ae = sample[sample['line_number']=='Part IX Line 24a - 24e; col (A)']
#get all rows for Line 24e
test_24f = sample[sample['line_number']=='Part IX Line 24f; col (A)']

# add all the values for 24a to 24e
for i in test_24f['object_id']:
    #set count to zero for each object_id
    count = 0 
    #add count of Line 24e
    count = count + test_24f['value'].where(test_24f['object_id']==i).sum()
    #add count of Line 24a-d (4 lines)
    count = count + test_24ae['value'].where(test_24ae['object_id']==i).sum()
    #get index of object_id
    i_index = test_24f[test_24f['object_id']==i].index
    #update sample with total value at Line 24e
    sample.at[i_index,'value'] = count
    #rename line 24e to Line 24com
    sample.at[i_index,'line_number'] = 'Part IX Line 24com'

#drop Line 24a - 24e for rows with added sum in Line 24f
for i in test_24f['object_id']:
    loc_24 = sample[sample['object_id']==i]
    loc_index = loc_24[loc_24['line_number'] =='Part IX Line 24a - 24e; col (A)'].index
    sample.drop(loc_index, inplace=True)

In [11]:
#get all rows for Line 24a - 24e that did not have 24f
test_24aeNOTf = sample[sample['line_number']=='Part IX Line 24a - 24e; col (A)']

# add all the values for 24a to 24e
for i in test_24aeNOTf['object_id']:
    #set count to zero for each object_id
    count = 0 
    #add count of Line 24a-e
    count = count + test_24aeNOTf['value'].where(test_24aeNOTf['object_id']==i).sum()
    #get index of object_id
    i_index = test_24aeNOTf[test_24aeNOTf['object_id']==i].index
    #update sample with total value
    sample.at[i_index,'value'] = count
    #change the name of the file line to Line 24com
    sample.at[i_index[0],'line_number'] = 'Part IX Line 24com'


In [12]:
#get list of Line 24com
test_24com = sample[sample['line_number']=='Part IX Line 24com']

#drop Line 24a - 24e for rows with added sum in Line 24com
for i in test_24com['object_id']:
    l_24 = sample[sample['object_id']==i]
    l_index = l_24[l_24['line_number'] =='Part IX Line 24a - 24e; col (A)'].index
    sample.drop(l_index, inplace=True)

In [13]:
#get list of Line 24 combined 
test_24com = sample[sample['line_number']=='Part IX Line 24com']

#replace all Part IX Line 24com description's with "All other expenses"
sample.at[test_24com.index,'description'] = "All other expenses"
    


In [14]:
#look at descriptions
sorted(sample['description'].unique())

['Advertising; Total',
 'All other expenses',
 'Benefits paid to or for members; Total',
 'Compensation and other distributions not included above to disqualified persons; Total',
 'Compensation of current officers, directors, and key employees; Total',
 'Conferences, conventions and meetings; Total',
 'Depreciation, depletion, and amortization; Total',
 'Fees for services (non-employees): Investment management fees; Total',
 'Fees for services (non-employees): Legal; Total',
 'Fees for services (non-employees): Lobbying; Total',
 'Fees for services (non-employees): Management; Total',
 'Fees for services (non-employees): Other; Total',
 'Fees for services (non-employees): Professional fundraising; Total',
 'Fees for services (non-employees): accounting; Total',
 'Grants and other assistance to governments, organizations and individuals outside the U.S.; Total',
 'Grants and other assistance to individuals in the U.S. Complete Parts II and III of Schedule I if total exceeds $5,000; Tot

In [15]:
#remove "Total" suffix from the descriptions
sample['description'] = sample['description'].str.replace('; Total', '')

#get index for 'Fees for services (non-employees)'
Imf_index = sample[sample['description']=='Fees for services (non-employees): Investment management fees'].index
legal_index = sample[sample['description']=='Fees for services (non-employees): Legal'].index
lobbying_index = sample[sample['description']=='Fees for services (non-employees): Lobbying'].index
management_index = sample[sample['description']=='Fees for services (non-employees): Management'].index
other_index = sample[sample['description']=='Fees for services (non-employees): Other'].index
pf_index = sample[sample['description']=='Fees for services (non-employees): Professional fundraising'].index
accounting_index = sample[sample['description']=='Fees for services (non-employees): accounting'].index

#replace label
sample.at[Imf_index,'description'] = 'Investment management fees'
sample.at[legal_index,'description'] = 'Legal fees'
sample.at[lobbying_index,'description'] = 'Lobbying fees'
sample.at[management_index,'description'] = 'Management fees'
sample.at[other_index,'description'] = 'Other fees'
sample.at[pf_index,'description'] = 'Professional fundraising fees'
sample.at[accounting_index,'description'] = 'Accounting fees'

In [16]:
#get all rows for Grants and other assistance to individuals in the U.S
grant_ind = sample[sample['description'].str.contains('Grants and other assistance to individuals in the U.S')]
#get all rows for Grants to governments and organizations in the U.S
grant_go = sample[sample['description'].str.contains('Grants to governments and organizations in the U.S')]

# add all the values for Grants in the U.S.
for i in grant_go['object_id']:
    # set sum to zero for each object_id
    value_sum = 0 
    # add value of individual grant
    value_sum = value_sum + grant_ind['value'].where(grant_ind['object_id']==i).sum()
    # add value of government grant
    value_sum = value_sum + grant_go['value'].where(grant_go['object_id']==i).sum()
    # get index of grant_go
    go_index = grant_go[grant_go['object_id']==i].index
    #update sumed value
    sample.at[go_index,'value'] = value_sum
    #rename grant description
    sample.at[go_index,'description'] = 'Grants and other assistance to governments, organizations and individuals in the U.S'
    

In [17]:
#get list of summed grants in the U.S.
line_1 = sample[(sample['line_number']=='Part IX Line 1; col (A)')]

#drop lines that have individual grants,
for i in line_1['object_id']:
    line_samp = sample[(sample['object_id']==i)]
    line_2index = line_samp[(line_samp['line_number']=='Part IX Line 2; col (A)')].index
    sample.drop(line_2index, inplace=True)


In [18]:
# get list of remaining individual grants that didn't have line_1
line_2 = sample[(sample['line_number']=='Part IX Line 2; col (A)')]

# rename grant description
for i in line_2['object_id']:
    line2_samp = sample[(sample['object_id']==i)]
    line2_index = line2_samp[(line2_samp['line_number']=='Part IX Line 2; col (A)')].index
    sample.at[line2_index,'description'] = 'Grants and other assistance to governments, organizations and individuals in the U.S'


In [19]:
#get indexes of grants in U.S.
len_grant = sample[(sample['description']=='Grants and other assistance to governments, organizations and individuals in the U.S')].index

# rename lines
sample.at[len_grant,'line_number'] = 'Part IX Line 1/2'
    

In [68]:
#remove col(A) from from lines
sample['line_number'] = sample['line_number'].str.replace('; col \(A\)', '')


<ipython-input-68-730069f97074>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sample['line_number'] = sample['line_number'].str.replace('; col \(A\)', '')


In [69]:
#save date as csv
sample.to_csv("cleaned_data.csv",index=False)

In [75]:
data = pd.read_csv("cleaned_data.csv")
#check over hospital names
sorted(data['name'].unique())


['200 HOSPITAL - MYMC INC',
 'A3R LLC DBA Vitel Net',
 'ABBEVILLE COUNTY MEMORIAL HOSPITAL',
 'ABBEVILLE COUNTY MEMORIAL HOSPITAL FOUNDATION',
 'ABBOTT NORTHWESTERN HOSPITAL FOUNDATION',
 'ABINGTON MEMORIAL HOSPITAL',
 'ABROM KAPLAN MEMORIAL HOSPITAL',
 'ACADIA GENERAL HOSPITAL INC',
 'ACC Pharmacy',
 'ACCOUNTING DEPARTMENT',
 'ACCRETIVEHEALTH INC',
 'ADAIR COUNTY HOSPITAL DISTRICT',
 'ADAMS COUNTY MEMORIAL HOSPITAL AND',
 'ADELE DUCHARME',
 'ADVANCE CARE HOSPITAL',
 'ADVANCED ANESTHESIA',
 'ADVANCED CARE HOSPITAL OF WHITE COUNTY',
 'ADVENTHEALTH DELAND',
 'ADVENTHEALTH DURAND',
 'ADVENTHEALTH HENDERSONVILLE',
 'ADVENTHEALTH MANCHESTER',
 'ADVENTHEALTH PALM COAST',
 'ADVENTHEALTH PALM COAST AUXILIARY',
 'ADVENTHEALTH ZEPHYRHILLS',
 'ADVENTIST HEALTHCARE REHABILITATION',
 'ADVENTIST HEALTHCARE SHADY GROVE',
 'ADVENTIST HEALTHCARE WHITE OAK',
 'ADVENTIST REHABILITATION HOSPITAL OF MARYLAND',
 'ADVOCATE AURORA HEALTH INC',
 'ADVOCATE SHERMAN HOSPITAL',
 'ADVOCATES FOR UPSTATE MEDICAL UNIV

In [202]:
#creat set for names that contain 'hospital'
hospital = set()
#creat set for names that don't contain 'hospital'
non_hospital = set()

#add to above sets the correct values
for i in data['name']:
    if 'hospital' in i.lower():
        hospital.add(i)
    else:
        non_hospital.add(i)

#get object id of the non hospitals        
non_id = set()       
for i in non_hospital:
    non_index = data[data['name'] == i].index
    for j in non_index:
        non_id.add(data.at[j,'object_id'])


In [229]:
import os

test_data = data.copy()

#open each csv file and cut part IX
path = '/Users/vidhipatel/Desktop/990 Project/data'
for filename in os.listdir(path):
    for i in non_id:
        if filename == (str(i)+'.csv'):
            with open(os.path.join(path, filename)) as f:
                # read file as csv
                content = pd.read_csv(f)
                # get rows with business name
                nameline = content[content["description"]
                            .str.contains("business name line ",na=False, case=False)==True]
                nameline = nameline.reset_index()
                # get rows with hospital name
                name = nameline[nameline["value"]
                            .str.contains("hospital",na=False, case=False)==True]
                name = name.reset_index()
                #get hospital name
                if (len(name.index)) >= 1:
                    hos_name = name.loc[0,'value']
                else:
                    hos_name = name['value']

                #add name to datafile
                data_index = test_data[test_data['object_id'] == i].index
                test_data.at[data_index,'name'] = hos_name
                    

In [244]:
test_data['name'] = test_data['name'].astype(str)

In [258]:
#creat set for names that contain 'hospital'
hospital_test = set()
#creat set for names that don't contain 'hospital'
non_hospital_test = set()

#add to above sets the correct values
for i in test_data['name']:
    if 'hospital' in i.lower():
        hospital_test.add(i)
    else:
        non_hospital_test.add(i)

#get object id of the non hospitals        
non_id_test = set()       
for i in non_hospital_test:
    non_index = test_data[test_data['name'] == i].index
    for j in non_index:
        non_id_test.add(test_data.at[j,'object_id'])
        
print(non_hospital_test)

{'nan'}


In [259]:
test_data[test_data['name']=='nan']

,Unnamed: 0,object_id,line_number,description,value,name
110,151,202043389349301214,Part IX Line 1/2,"Grants and other assistance to governments, or...",10015,nan
111,153,202043389349301214,Part IX Line 3,"Grants and other assistance to governments, or...",0,nan
112,154,202043389349301214,Part IX Line 4,Benefits paid to or for members,0,nan
113,155,202043389349301214,Part IX Line 5,"Compensation of current officers, directors, a...",0,nan
114,156,202043389349301214,Part IX Line 6,Compensation and other distributions not inclu...,0,nan
...,...,...,...,...,...,...
470121,638115,201922269349301307,Part IX Line 13,Office expenses,21985,nan
470122,638116,201922269349301307,Part IX Line 19,"Conferences, conventions and meetings",20400,nan
470123,638117,201922269349301307,Part IX Line 22,"Depreciation, depletion, and amortization",1487,nan
470124,638118,201922269349301307,Part IX Line 23,Insurance,1823,nan


In [206]:
import os

#open each csv file and cut part IX
path = '/Users/vidhipatel/Desktop/990 Project/data'
for filename in os.listdir(path):
    for i in non_id_test:
        if filename == (str(i)+'.csv'):
            with open(os.path.join(path, filename)) as f:
                # read file as csv
                content = pd.read_csv(f)
                # get rows with business name
                nameline = content[content["description"]
                            .str.contains("business name line ",na=False, case=False)==True]
                nameline = nameline.reset_index()
                com_name = nameline.at[0,'value'] +" "+ nameline.at[1,'value']
                #add name to datafile
                data_index = test_data[test_data['object_id'] == i].index
                test_data.at[data_index,'name'] = com_name

In [216]:
#creat set for names that contain 'hospital'
hospital_test2 = set()
#creat set for names that don't contain 'hospital'
non_hospital_test2 = set()

#add to above sets the correct values
for i in test_data['name']:
    if 'hospital' in i.lower():
        hospital_test2.add(i)
    else:
        non_hospital_test2.add(i)

#get object id of the non hospitals        
non_id_test2 = set()       
for i in non_hospital_test2:
    non_index = test_data[test_data['name'] == i].index
    for j in non_index:
        non_id_test2.add(test_data.at[j,'object_id'])


In [223]:
#open each csv file and cut part IX
path = '/Users/vidhipatel/Desktop/990 Project/data'
for filename in os.listdir(path):
    for i in non_id_test2:
        if filename == (str(i)+'.csv'):
            with open(os.path.join(path, filename)) as f:
                # read file as csv
                content = pd.read_csv(f)
                # get rows with business name
                nameline = content[content["description"]
                            .str.contains("business name line ",na=False, case=False)==True]
                nameline = nameline.reset_index()
                 # get rows with hospital name
                name = nameline[nameline["value"]
                            .str.contains("hospital",na=False, case=False)==True]
                name = name.reset_index()
                #combine name
                com_name = nameline.at[0,'value'] +" "+ nameline.at[1,'value']
        break           
#                 #add name to datafile
#                 data_index = test_data[test_data['object_id'] == i].index
#                 test_data.at[data_index,'name'] = com_name
print(name)
print(com_name)

   level_0  index           object_id          form line_number  \
0        2     11  201643199349307394  ReturnHeader         NaN   

            description                                value  \
0  Business name line 2  DBA Brookside Convalescent Hospital   

                                   variable_name  \
0  returnheader990x_part_i.BsnssNm_BsnssNmLn2Txt   

                                               xpath  in_group group_name  \
0  /ReturnHeader/Filer/BusinessName/BusinessNameL...     False        NaN   

   group_index  
0          NaN  
Hansen Hunter & Co PC St Johns Volunteers


In [177]:
sorted(non_hospital_test)       

['ADVOCATES FOR UPSTATE MEDICAL UNIVERSITY',
 'AMBASSADOR CLUB OF HENRY FORD',
 'AMELIA BROWN FRAZIER REHAB CTR INC BOARD DIR',
 'AMERICAN COMMITTEE FOR SHAARE ZEDEK',
 'AMERICAN FOUNDATION FOR ST GEORGE',
 'AMERICAN FRIENDS OF KIRYAT SANZ',
 'AMERICAN FRIENDS OF WGH WESTERN GALILEE',
 'AMERICUS AND SUMTER COUNTY',
 'ARLENE A SNYDER',
 'AUX TO SENTARA VA BEACH GENERAL',
 'AUXILIARY BOARD OF NORTHWESTERN',
 'AUXILIARY BOARD TO VIRTUA-MEMORIAL',
 'AUXILIARY OF JOHN H STROGER JR',
 'AUXILIARY OF THE CAMDEN CLARK',
 'AUXILIARY OF THE CAMDEN CLARK MEMORIAL',
 'AUXILIARY OF THE NEMOURSALFRED I DUPONT',
 'AUXILIARY OF THE RIVERSIDE COMMUNITY',
 'AUXILIARY OF THE ROBERT WOOD JOHNSON',
 'AUXILIARY TO SENTARA VIRGINIA BEACH',
 'Anne Ford',
 'Aurora Health Care Inc',
 'Auxiliary of Riverside Shore',
 'Auxiliary of the Camden Clark Memorial',
 'BEHAVIORAL HEALTH PARTNERS OF FREDERICK',
 'BLUE BIRD AUXILIARY OF THE SOUTHWEST',
 'BOWLING GREEN WARREN COUNTY COMMUNITY',
 'Bayhealth Medical Center Inc

In [207]:
test_data[test_data['object_id'] == 201323199349303297]

,Unnamed: 0,object_id,line_number,description,value,name
15843,21533,201323199349303297,Part IX Line 1/2,"Grants and other assistance to governments, or...",406850,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15844,21535,201323199349303297,Part IX Line 13,Office expenses,98578,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15845,21536,201323199349303297,Part IX Line 16,Occupancy,780,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15846,21537,201323199349303297,Part IX Line 17,Travel,6165,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15847,21538,201323199349303297,Part IX Line 19,"Conferences, conventions and meetings",16271,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15848,21539,201323199349303297,Part IX Line 22,"Depreciation, depletion, and amortization",648,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15849,21542,201323199349303297,Part IX Line 24com,All other expenses,806143,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
15850,21543,201323199349303297,Part IX Line 25,Total functional expenses,1335435,LOMA LINDA UNIVERSITY CHILDREN'S HOSPITAL FOUN...
